In [4]:
import yfinance as yf
import pandas as pd

ticker = "AAPL"  # replace with your ticker
stock = yf.Ticker(ticker)

ohlc_df = stock.history(period="max", interval="1d")[['Open', 'High', 'Low', 'Close']]
ohlc_df.index = ohlc_df.index.tz_localize(None)  # Make index timezone-naive

income_stmt = stock.financials.T  # Transpose so each row is a report
income_stmt.index = pd.to_datetime(income_stmt.index)  # Ensure datetime index

# Create a DataFrame with only dates and initialize empty columns
dates_df = pd.DataFrame(index=ohlc_df.index)

# Combine: For each income statement report, assign it to the days after it
income_stmt_sorted = income_stmt.sort_index()
aligned_income = pd.DataFrame(index=ohlc_df.index, columns=income_stmt.columns)

for report_date in income_stmt_sorted.index:
    mask = (aligned_income.index >= report_date)
    aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
    aligned_income.loc[aligned_income.index >= report_date] = \
    pd.DataFrame([income_stmt.loc[report_date].values] * len(aligned_income.loc[aligned_income.index >= report_date]),
                 index=aligned_income.loc[aligned_income.index >= report_date].index,
                 columns=aligned_income.columns)


# Now forward-fill missing days (between reports)
aligned_income.fillna(method='ffill', inplace=True)

# Final combined DataFrame
combined_df = pd.concat([ohlc_df, aligned_income], axis=1)

combined_df

C:\Users\dawid\AppData\Local\Temp\ipykernel_16484\3801307920.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\Temp\ipykernel_16484\3801307920.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\Temp\ipykernel_16484\3801307920.py:22: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  aligned_income.loc[mask] = aligned_income.loc[mask].fillna(method='ffill')
C:\Users\dawid\AppData\Local\

,Open,High,Low,Close,Tax Effect Of Unusual Items,Tax Rate For Calcs,Normalized EBITDA,Net Income From Continuing Operation Net Minority Interest,Reconciled Depreciation,Reconciled Cost Of Revenue,...,Interest Expense Non Operating,Interest Income Non Operating,Operating Income,Operating Expense,Research And Development,Selling General And Administration,Gross Profit,Cost Of Revenue,Total Revenue,Operating Revenue
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-12,0.098597,0.099025,0.098597,0.098597,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-15,0.093881,0.093881,0.093453,0.093453,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-16,0.087022,0.087022,0.086594,0.086594,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-17,0.088737,0.089165,0.088737,0.088737,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980-12-18,0.091309,0.091738,0.091309,0.091309,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-09,198.739390,200.277366,197.281295,198.270004,0.0,0.241,1.346610e+11,9.373600e+10,1.144500e+10,2.103520e+11,...,3.933000e+09,3.750000e+09,1.232160e+11,5.746700e+10,3.137000e+10,2.609700e+10,1.806830e+11,2.103520e+11,3.910350e+11,3.910350e+11
2025-05-12,210.970001,211.270004,206.750000,210.789993,0.0,0.241,1.346610e+11,9.373600e+10,1.144500e+10,2.103520e+11,...,3.933000e+09,3.750000e+09,1.232160e+11,5.746700e+10,3.137000e+10,2.609700e+10,1.806830e+11,2.103520e+11,3.910350e+11,3.910350e+11
2025-05-13,210.429993,213.399994,209.000000,212.929993,0.0,0.241,1.346610e+11,9.373600e+10,1.144500e+10,2.103520e+11,...,3.933000e+09,3.750000e+09,1.232160e+11,5.746700e+10,3.137000e+10,2.609700e+10,1.806830e+11,2.103520e+11,3.910350e+11,3.910350e+11
